In [1]:
%matplotlib inline

# Spike Sorting shuttle drive zero to hero

In [5]:
import os
from pathlib import Path

from spikeinterface.extractors import read_neuralynx, NeuralynxRecordingExtractor
import spikeinterface as si  # import core only
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw

import numpy as np
import scipy as sp
import scipy.optimize
import matplotlib.pyplot as plt

from probeinterface import Probe, ProbeGroup
from probeinterface.plotting import plot_probe, plot_probe_group

from probeinterface import generate_tetrode

ModuleNotFoundError: No module named 'spikeinterface.toolkit'

In [4]:
ROOT = Path(r"E:\neuralynx").resolve()

Setting KILOSORT3_PATH environment variable for subprocess calls to: C:\Users\Dejana\Documents\GitHub\Kilosort-2.5


### Define constants

In [ ]:
BAD_CHANNELS = ()
BAD_TETRODE = ()

NUMBER_OF_TETRODES = 4

SCALE = 1000
RADIUS_MIN = 0.1 * SCALE
RADIUS_MAX = 0.15 * SCALE
HALF_CIRCLE = False

MOUSE_ID = "668"
RECORDING_ID = "2023-02-01_13-04-03"

RECORDING_PATH = ROOT / MOUSE_ID / RECORDING_ID
recording = read_neuralynx(r'E:\neuralynx\668\2023-02-01_13-04-03')
print(recording)

In [ ]:
recording = si.concatenate_recordings([recording])

In [ ]:
print(recording.channel_ids)

In [ ]:
number_of_channels = NUMBER_OF_TETRODES * 4

#recording_waveform_path = WAVEFORM_PATH / RECORDING_ID

### Define n number of points along the elipse
Source: https://stackoverflow.com/questions/6972331/how-can-i-generate-a-set-of-points-evenly-distributed-along-the-perimeter-of-an

In [ ]:
def angles_in_ellipse(
        num,
        a,
        b):
    assert(num > 0)
    assert(a < b)
    angles = 2 * np.pi * np.arange(num) / num
    if a != b:
        e = (1.0 - a ** 2.0 / b ** 2.0) ** 0.5
        tot_size = sp.special.ellipeinc(2.0 * np.pi, e)
        arc_size = tot_size / num
        arcs = np.arange(num) * arc_size
        res = sp.optimize.root(
            lambda x: (sp.special.ellipeinc(x, e) - arcs), angles)
        angles = res.x 
    return angles


a = RADIUS_MIN
b = RADIUS_MAX
n = NUMBER_OF_TETRODES * 2 if HALF_CIRCLE else NUMBER_OF_TETRODES

phi = angles_in_ellipse(n, a, b)
tetrode_positions = np.array((b * np.sin(phi), a * np.cos(phi))).T

# # plotting
# import matplotlib.pyplot as plt
# fig = plt.figure()
# ax = fig.gca()
# ax.axes.set_aspect('equal')
# ax.scatter(b * np.sin(phi), a * np.cos(phi))
# plt.show()

### Generate probegroup for use in spike sorting

In [ ]:
probegroup = ProbeGroup()

for i in range(NUMBER_OF_TETRODES):
    tetrode = generate_tetrode()
    tetrode.move(tetrode_positions[i])
    probegroup.add_probe(tetrode)

#t_1 = [16, 17, 18, 19]
#t_2 = [20, 21, 22, 23]
#t_3 = [24, 25, 26, 27]
#t_4 = [28, 29, 30, 31]

t_1 = [0,1,2,3]
t_2 = [4,5,6,7]
t_3 = [8,9,10,11]
t_4 = [12,13,14,15]

probegroup.set_global_device_channel_indices(np.array(t_1 + t_2 + t_3 + t_4))
print(probegroup)

In [ ]:
plot_probe_group(probegroup, with_channel_index=True, same_axes=True)

## Spike sorting time!

In [ ]:
#recording = se.read_openephys(OE_RECORDING_PATH)
recording.set_probegroup(probegroup, in_place=True)

keep_ids = [recording.channel_ids[i] for i in range(number_of_channels) if i not in BAD_CHANNELS]
recording = recording.channel_slice(keep_ids)

### Filter your data to only include spikes

In [ ]:
from spikeinterface.preprocessing import bandpass_filter

recording = bandpass_filter(recording, freq_min=300, freq_max=6000)

In [ ]:
import spikeinterface.sorters as ss

VERBOSE = False
PATH_TO_OUTPUT_FOLDER = ""
SORTER_PARAMS = {

}

kilosort_sorter_class = ss.Kilosort3Sorter
kilosort_sorter_class.set_kilosort3_path(r"C:\Users\Dejana\Documents\GitHub\Kilosort")

output_folder = kilosort_sorter_class.initialize_folder(
    recording, PATH_TO_OUTPUT_FOLDER, VERBOSE, remove_existing_folder=True)
kilosort_sorter_class.set_params_to_folder(
    recording, output_folder, SORTER_PARAMS, VERBOSE)
kilosort_sorter_class.setup_recording(recording, output_folder, verbose=VERBOSE)

# We run the sorter here!
kilosort_sorter_class.run_from_folder(output_folder, True, VERBOSE)

sorting = kilosort_sorter_class.get_result_from_folder(output_folder)

sorter_output_folder = output_folder / "sorter_output"

In [ ]:
import spikeinterface as si
from spikeinterface.exporters import export_to_phy

phy_folder = Path(r'E:\neuralynx\Data_acquired_2022-08-16_2022-08-22\phy_data')
output_folder = phy_folder / RECORDING_ID

we = si.extract_waveforms(recording, sorter, output_folder)

In [ ]:
phy_output_folder = output_folder / 'phy'
export_to_phy(we, phy_output_folder)